### First Pass
- take care of nulls
- data errors
- data types
- dummy vars
- split
- scaling
- features (select kbest, recursive feature engineering)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("student/student-mat.csv", sep=";")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [4]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


**Object Columns**

How many unique values in each column? We need to answer this so that we know if creating dummy variables makes sense (or if it ends up creating way too many columns).
- Create a boolean mask of the columns indicating whether the datatype is object or not.

In [5]:
# df.dtypes == 'object' returns a series. 
# convert this to an array
mask = np.array(df.dtypes == "object")

- filter the dataframe columns by using the mask

In [6]:
# using iloc, the df will filter out all the index locations 
# (columns number) where mast is false 

obj_df = df.iloc[:, mask]

- loop through all the object columns and generate value counts of each unique value.

In [7]:
# loop through each column name in the list of columns
# print the value_counts 

for col in obj_df.columns:
    print(obj_df[col].value_counts())
    print("\n")

GP    349
MS     46
Name: school, dtype: int64


F    208
M    187
Name: sex, dtype: int64


U    307
R     88
Name: address, dtype: int64


GT3    281
LE3    114
Name: famsize, dtype: int64


T    354
A     41
Name: Pstatus, dtype: int64


other       141
services    103
at_home      59
teacher      58
health       34
Name: Mjob, dtype: int64


other       217
services    111
teacher      29
at_home      20
health       18
Name: Fjob, dtype: int64


course        145
home          109
reputation    105
other          36
Name: reason, dtype: int64


mother    273
father     90
other      32
Name: guardian, dtype: int64


no     344
yes     51
Name: schoolsup, dtype: int64


yes    242
no     153
Name: famsup, dtype: int64


no     214
yes    181
Name: paid, dtype: int64


yes    201
no     194
Name: activities, dtype: int64


yes    314
no      81
Name: nursery, dtype: int64


yes    375
no      20
Name: higher, dtype: int64


yes    329
no      66
Name: internet, dtype: int64


no    

In [8]:
df.nunique()

school         2
sex            2
age            8
address        2
famsize        2
Pstatus        2
Medu           5
Fedu           5
Mjob           5
Fjob           5
reason         4
guardian       3
traveltime     4
studytime      4
failures       4
schoolsup      2
famsup         2
paid           2
activities     2
nursery        2
higher         2
internet       2
romantic       2
famrel         5
freetime       5
goout          5
Dalc           5
Walc           5
health         5
absences      34
G1            17
G2            17
G3            18
dtype: int64

**Dummy Variables**

In [9]:
# create df with new dummy vars
dummy_df = pd.get_dummies(obj_df, dummy_na=False, drop_first=True)

In [10]:
# concatenate the dataframe with dummies to our original dataframe
# via column (axis=1)
df = pd.concat([df, dummy_df], axis=1)

In [11]:
# drop object columns from df
df.drop(columns=obj_df.columns, inplace=True)

**Split**

In [12]:
from sklearn.model_selection import train_test_split

train_validate, test = train_test_split(df,
                                        test_size=.2,
                                        random_state=123)
train, validate = train_test_split(train_validate, 
                                   test_size=.3,
                                   random_state=123)

**Split into X and y dataframes**
- y = G3

In [13]:
# x df's are all cols except G3
X_train = train.drop(columns=['G3'])
X_validate = validate.drop(columns=['G3'])
X_test = test.drop(columns=['G3'])

# y df's are just G3
y_train = train[['G3']]
y_validate = validate[['G3']]
y_test = test[['G3']]

**Scale**

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(copy=True)
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

Create dataframes out of the scaled arrays that were generated by the scaler tranform.

In [15]:
X_train_scaled = pd.DataFrame(X_train_scaled, 
             columns = X_train.columns.values).\
             set_index(X_train.index.values)

X_validate_scaled = pd.DataFrame(X_validate_scaled, 
             columns = X_validate.columns.values).\
             set_index(X_validate.index.values)

X_test_scaled = pd.DataFrame(X_test_scaled, 
             columns = X_test.columns.values).\
             set_index(X_test.index.values)

##### Feature Selection
1. SelectKBest
2. RFE: Recursive Feature Elimination

##### SelectKBest
- filter method
- find and keep the attributes with the highest correlation to the target variable.

How?
1. the correlation between each attribute & the target is computed.
2. converted to an F-score and then p-value.
3. top k attributes are kept.

In [16]:
from sklearn.feature_selection import SelectKBest, f_regression

- Initialize the f_selector object, defining the scoring method

In [17]:
f_selector = SelectKBest(f_regression, k = 13)

- Fit the object to our X and y data(train). This will score, rank and ID the top k feature

In [18]:
f_selector = f_selector.fit(X_train_scaled, y_train.G3)

- Transform our dataset to reduct to the K Best features

In [19]:
X_train_reduced = f_selector.transform(X_train_scaled)

print(X_train.shape)
print(X_train_reduced.shape)

(221, 41)
(221, 13)


In [20]:
f_support = f_selector.get_support()
type(f_support)

numpy.ndarray

Create a dataframe with just the selected features.

In [30]:
# using iloc, the df will filter out all the index locations 
# (columns number) where mask is false
# the : before the comma is for rows (so if we wanted to filter rows 
# we could say like 10:20), and after the comma is for columns. 

X_reduced_scaled = X_train_scaled.iloc[:,f_support]

This new dataframe is ready for modeling!

X_reduced_scaled.head()

View the features selected:

In [23]:
f_feature = X_train_scaled.iloc[:, f_support].columns.tolist()
f_feature

['age',
 'Medu',
 'Fedu',
 'traveltime',
 'studytime',
 'failures',
 'G1',
 'G2',
 'sex_M',
 'Mjob_other',
 'reason_reputation',
 'guardian_other',
 'higher_yes']

We could run through again with a different k value and select those best features. We can then run the different dataframes through models and select the best model

**Recursive Feature Engineering: RFE**

wrapper method

Recursively build model after model with fewer and fewer features. It will then identify which model performs the best. Then, return which features were used in that model. Those are the features we will keep.

In [24]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

Initialize the linear regression object

In [25]:
lm = LinearRegression()

Initialize the RFE object, setting the hyperparameters to be our linear model above(lm) and the number of features we want returned.

In [26]:
rfe = RFE(lm, 13)

In [27]:
X_rfe = rfe.fit_transform(X_train_scaled, y_train.G3)

In [31]:
mask = rfe.support_

In [32]:
X_reduced_scaled_rfe = X_train_scaled.iloc[:, mask]

In [33]:
# features selected using rfe
X_reduced_scaled_rfe.columns.tolist()

['age',
 'traveltime',
 'failures',
 'famrel',
 'absences',
 'G1',
 'G2',
 'Mjob_health',
 'Mjob_other',
 'Mjob_services',
 'schoolsup_yes',
 'famsup_yes',
 'internet_yes']

In [34]:
# features selected using selectkbest
X_reduced_scaled.columns.tolist()

['age',
 'Medu',
 'Fedu',
 'traveltime',
 'studytime',
 'failures',
 'G1',
 'G2',
 'sex_M',
 'Mjob_other',
 'reason_reputation',
 'guardian_other',
 'higher_yes']